In [1]:
import constants as c
import functions
import optimizing
import preprocessing
import testreferenceyears
from visualize import *
from forecasting import *

import os
import io
from ipywidgets import Button
import asyncio
from ipywidgets import IntSlider, FloatSlider, FloatText, Text, widgets, Checkbox, FileUpload, ToggleButtons
from IPython.display import display

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
M_eta_new, P_e_new, P_w_new, data_temp_new = None, None, None, None

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True)
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True`
    widget.on_click(getvalue)
    return future

layout = widgets.Layout(width='auto', height='40px')

room_temp_w = FloatSlider(value=21, min=17, max=27, step = 0.5, description= 'Desired room temperature [°C]: ', continuous_update = False, layout=layout, style= {'description_width': 'initial'})
return_flow_w = FloatSlider(value=25, min=21, max=50, step = 0.5, description= 'Returning flow temperature [°C]: ', continuous_update = False, layout=layout, style= {'description_width': 'initial'})
ny_w = FloatSlider(value=0.4, min=0.1, max=1, step = 0.1, description= 'Quality factor of the heat pump: ', continuous_update = False, layout=layout, style= {'description_width': 'initial'})

flow_max_w = FloatText(value= 50, description = 'Maximum flow temperature at minimum temperature [°C]: ', layout=layout, style= {'description_width': 'initial'})

out_min_w = FloatText(value= -15, description = 'Minimum outside temperature [°C]: ', layout=layout, style= {'description_width': 'initial'})

house_energy_w = FloatText(value= 200, description = 'Energy need in Watt per Kelvin [W/K]: ', layout=layout, style= {'description_width': 'initial'}) # Watt / Kelvin
storage_power_w = FloatText(value= 200, description = 'Energy need of your storage (DIN...): ', layout=layout, style= {'description_width': 'initial'}) #Watts
storage_volume_w = FloatText(value= 4000, description = 'Storage volume [Liter or equivalent]: ', layout=layout, style= {'description_width': 'initial'}) #Liter = kilogram -> to be defined later on

hp_power_el_opt_w = FloatText(value= 5000, description = 'Power of heat pump [Watt]: ', layout=layout, style= {'description_width': 'initial'}) #Watts -> Power in which heatpump works most efficiently (can be taken into calculation later)
max_heat_cap_w = IntSlider(value=80, description='Maximum heating temperature [°C]: ', min=50, max=95, layout=layout, style= {'description_width': 'initial'}) # temperature, up to which the heat pump can maximally heat

address_w = Text(placeholder = 'Examplestreet 1, 12345 Examplecity', description = 'Address: ', layout=layout, style= {'description_width': 'initial'})


toggle_forecast = ToggleButtons(options=['Whole year', 'Forecast'], description = 'Type of calculation: ', tooltips = ['Take TRY data or similar for overall saving potentials.', 'Optimize energy use for future periods.'], layout=layout, style= {'description_width': 'initial'})
check_vis = Checkbox(False, description = 'Generate visualizations?')
check_opt = Checkbox(True, description = 'Generate optimization?')
check_owndata = Checkbox(False, description='Upload own temperature data?')

upload_w = FileUpload(description = 'Upload')

button_w = Button(description="GO!")

first_round = True
def main_func():
    global M_eta_new, P_e_new, P_w_new, data_temp_new, first_round

    room_temp = room_temp_w.value
    return_flow = return_flow_w.value
    ny = ny_w.value
    house_energy = house_energy_w.value
    storage_power = storage_power_w.value
    storage_volume = storage_volume_w.value
    hp_power_el_opt = hp_power_el_opt_w.value
    max_heat_cap = max_heat_cap_w.value
    address = address_w.value

    flow_max = flow_max_w.value
    out_min = out_min_w.value

    # TODO choose between Forecasting and StationData
    funs = functions.Functions(flow_max=flow_max, room_temp=room_temp, out_min=out_min, return_flow=return_flow, house_energy=house_energy, storage_volume=storage_volume, storage_power=storage_power, ny=ny)
    if toggle_forecast.value == 'Forecast':
        data_temp = get_weather_forecast(address)
    else:
        if upload_w.value == {}:
            data_temp = testreferenceyears.StationFinder(address).main()
        else:
            uploaded_file = upload_w.value
            data_temp = pd.read_csv(io.BytesIO(list(uploaded_file.values())[0]['content']), sep=';')
            data_temp.columns = ['date', 'temperature']
            data_temp.date = pd.to_datetime(data_temp.date)
            data_temp.temperature = data_temp.temperature.astype(float)
    preprocessor = preprocessing.Preprocessor(flow_max=flow_max, room_temp=room_temp, out_min=out_min, return_flow=return_flow, house_energy=house_energy, storage_volume=storage_volume, storage_power=storage_power, ny=ny)
    optimizer = optimizing.Optimizer(functions=funs, max_heat_cap=max_heat_cap, hp_power_el_opt=hp_power_el_opt, return_flow=return_flow, storage_volume=storage_volume)
    

    data_temp = preprocessor.add_data(data_temp)

    max_energy_el = data_temp.energy_needed_elec.max()
    if max_energy_el > hp_power_el_opt:
        print('Calculation not possible! Heat pump power not sufficient!')
        print('Continuing ...')

        return None, None, None, None

    data_temp = preprocessor.get_local_extrema(data_temp)
    if check_vis.value:
        temp_plot = plot_temperature(data_temp, address, True)

        if first_round:
            temp_plot.show()
            first_round = False

    if check_opt.value:
        M_eta_new, P_e_new, P_w_new, data_temp_new = optimizer.optimize_periods(data_temp)

        if not os.path.isdir(f"results/{address}"):
            os.makedirs(f'results/{address}')
        if not os.path.isdir(f"results/{address}/vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}"):
            os.makedirs(f'results/{address}/vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}')



        data_temp_new.to_csv(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\data_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.csv', sep=';', index=False)
        #M_eta_new.tofile(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\Eta_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.csv', sep=';')
        #P_e_new.tofile(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\Pe_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.csv', sep=';')
        #P_w_new.tofile(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\Pw_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.csv', sep=';')

        if check_vis.value:
            temp_plot.write_html(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\Temp_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html')

            opt_plot = plot_optimization(data_temp_new)
            opt_plot.write_html(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\Opt_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html', auto_open=False)

            M_eta_hm = plot_heatmap(M_eta_new)
            M_eta_hm.write_html(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\Etahm_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html', auto_open=False)

            P_e_hm = plot_heatmap(P_e_new)
            P_e_hm.write_html(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\Pehm_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html', auto_open=False)

            P_w_hm = plot_heatmap(P_w_new)
            P_w_hm.write_html(rf'.\results\{address}\vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}\Pwhm_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html', auto_open=False)

        print(''.center(40, '_'))
        print()
        print('Energy saving potentials: ', "{:10.2f}".format((1 - data_temp_new.P_e_opt.sum() / data_temp_new.energy_needed_elec.sum())*100), '%')
        print()
        print(''.center(40, '_'))
        print()
    print('DONE!')

    if check_opt.value:
        return M_eta_new, P_e_new, P_w_new, data_temp_new

async def f():
    x = await wait_for_change(button_w)
    print('------ computing saving potentials -------')
    main_func()

asyncio.create_task(f())

print('Adjust the values of the building and click GO! when you are ready.')

display(room_temp_w)
display(return_flow_w)
display(ny_w)
display(house_energy_w)
display(storage_power_w)
display(storage_volume_w)
display(hp_power_el_opt_w)
display(max_heat_cap_w)
display(address_w)
display(out_min_w)
display(flow_max_w)
display(toggle_forecast)
display(check_vis)
display(check_opt)
display(check_owndata)
upload_w.layout.visibility = "hidden"
display(upload_w)
display(button_w)

def appear_upload(box):
    if box['new']:
        upload_w.layout.visibility = "visible"
    else:
        upload_w.layout.visibility = "hidden"

def appear_owndata(box):

    if box['new'] == 'Forecast':
        upload_w.layout.visibility = "hidden"
        check_owndata.value = False
        check_owndata.layout.visibility = 'hidden'
    else:
        check_owndata.layout.visibility = 'visible'

toggle_forecast.observe(appear_owndata, names='value')
check_owndata.observe(appear_upload, names='value')

Adjust the values of the building and click GO! when you are ready.


FloatSlider(value=21.0, continuous_update=False, description='Desired room temperature [°C]: ', layout=Layout(…

FloatSlider(value=25.0, continuous_update=False, description='Returning flow temperature [°C]: ', layout=Layou…

FloatSlider(value=0.4, continuous_update=False, description='Quality factor of the heat pump: ', layout=Layout…

FloatText(value=200.0, description='Energy need in Watt per Kelvin [W/K]: ', layout=Layout(height='40px', widt…

...

Checkbox(value=False, description='Generate visualizations?')

Checkbox(value=True, description='Generate optimization?')

Checkbox(value=False, description='Upload own temperature data?')

FileUpload(value={}, description='Upload', layout=Layout(visibility='hidden'))

Button(description='GO!', style=ButtonStyle())

# Calculations for evaluation in thesis

In [ ]:
locations = ['Hamburg','Karlsruhe', 'Oberstdorf','Villingen-Schwenningen']#, #
houses = [(150, 40),(200, 60), (350, 75)] #
heat_pump_powers = [3000,5000,7000,10000]#,2000,
storage_volumes = [(100, 40), (300, 50), (600, 65), (1000, 80),(2000, 100), (3000, 110), (5000, 130),(8000, 145), (12000, 155)]#, (20000, 170)

for loc in locations:
    print('location: ', loc)
    for house in houses:
        print('house: ', house)
        for heat_pump_power in heat_pump_powers:
            print('hp_power: ', heat_pump_power)
            for vol in storage_volumes:
                path = rf'.\results\{loc}\vol_{int(vol[0])},pow_{int(heat_pump_power)},en_{int(house[0])}\data_ad_{loc},vol_{int(vol[0])},pow_{int(heat_pump_power)},en_{int(house[0])}.csv'
                if os.path.exists(path):
                    continue

                print('storage volume: ', vol[0], 'liters')

                storage_volume_w.value = vol[0]
                storage_power_w.value = vol[1]
                address_w.value = loc
                house_energy_w.value = house[0]
                flow_max_w.value = house[1]
                hp_power_el_opt_w.value = heat_pump_power

                M_eta_new, P_e_new, P_w_new, data_temp_new = main_func()

location:  Hamburg
house:  (150, 40)
hp_power:  3000
storage volume:  100 liters
period:  166  / 166
________________________________________

Energy saving potentials:        0.05 %

________________________________________

DONE!
storage volume:  300 liters
period:  166  / 166
________________________________________

Energy saving potentials:        0.57 %

________________________________________

DONE!
storage volume:  600 liters
period:  166  / 166
________________________________________

Energy saving potentials:        1.61 %

________________________________________

DONE!
storage volume:  1000 liters
period:  166  / 166
________________________________________

Energy saving potentials:        3.32 %

________________________________________

DONE!
storage volume:  2000 liters
period:  166  / 166
________________________________________

Energy saving potentials:        6.91 %

________________________________________

DONE!
storage volume:  3000 liters
period:  166  / 166
_

In [ ]:
storage_volume_w.value = 100000
storage_power_w.value = 0
address_w.value = 'Oberstdorf'
house_energy_w.value = 150
flow_max_w.value = 40
hp_power_el_opt_w.value = 100000

M_eta_new, P_e_new, P_w_new, data_temp_new = main_func()